<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# LinkedIn - Send followers demographic data to a Google Sheets spreadsheet
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Google%20Sheets/Google_Sheets_Send_data.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

**Tags:** "linkedin #googlesheets #gsheet #data #naas_drivers #demographics #content #snippet

**Author:** [Asif Syed](https://www.linkedin.com/in/asifsyd/)

Pre-requisite: Google Sheet needs to be shared with NAAS' service account (mentioned below)

🔗 naas-share@naas-gsheets.iam.gserviceaccount.com

Go to the Google Sheet -> File -> Share -> Share with others -> Enter the above service account.

## Input

### Import library

In [ ]:
from naas_drivers import gsheet
from naas_drivers import linkedin
import pandas as pd
import numpy as np
import naas

### Setup LinkedIn
<a href='https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75'>How to get your cookies ?</a>

In [ ]:
# LinkedIn cookies
LI_AT = naas.secret.get("LI_AT") or "ENTER_YOUR_COOKIE_HERE" # EXAMPLE : "AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2"
JSESSIONID = naas.secret.get("JSESSIONID") or "ENTER_YOUR_JSESSIONID_HERE" # EXAMPLE : "ajax:8379907400220387585"

### Setup Google Sheets
Share your spreadsheet with 🔗 naas-share@naas-gsheets.iam.gserviceaccount.com

In [ ]:
SPREADSHEET_URL =  "ENTER_YOUR_SPREADSHEET_URL_HERE"
SHEET_NAME = "ENTER_YOUR_SHEET_NAME_HERE"

## Model

### Connect to LinkedIn

In [ ]:
linkedin.connect(LI_AT, JSESSIONID)

### Connect to gsheet

In [ ]:
gsheet.connect(SPREADSHEET_URL)

### Get the profiles of last 'n' followers (n is the value assigned to limit parameter below)

The 'start' parameter below specifies the number of initial rows to be skipped. 

These initial rows refer to the most recent profiles, hence it is by default maintained as 0, but it could be customized as per the need. 

In [ ]:
df = linkedin.network.get_followers(start=0, limit=1)
profiles = df['PROFILE_URL']

### Get Industry name and country details for each profile

In [ ]:
df_identity = pd.DataFrame()
df1 = pd.DataFrame()
for counter, profile in enumerate(profiles):
    df1 = linkedin.profile.get_identity(profile_url=profile).loc[:,['PROFILE_ID', 'COUNTRY', 'INDUSTRY_NAME']]
    df_identity = df_identity.append(df1)

### Get work experience details for each profile

In [ ]:
df_resume = pd.DataFrame()
df1 = pd.DataFrame()
for counter,profile in enumerate(profiles):
    df1 = linkedin.profile.get_resume(profile_url = profile)
    df_resume = df_resume.append(df1)
df_resume = df_resume[df_resume['CATEGORY'] == 'Experience'].loc[:,['PROFILE_ID','TITLE','DATE_START','DATE_END','PLACE', 'FIELD']]

### Merge two data frames.

In [ ]:
df_final = pd.merge(df_identity, df_resume, how='inner', on='PROFILE_ID')
df_final

## Output

### Send the data

In [ ]:
gsheet.send(
    sheet_name=SHEET_NAME,
    data=df_final
)